In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
from scipy import optimize
from numpy import random
import csv 

### Functions: square()

In [2]:
def square(points, delay, rise_time, on_time, fall_time, offset, amplitude):
    """points: total no of points
       delay: after how many points the pulse starts
       rise_time: in units of number of points
       on_time: in units of number of points
       fall_time: in units of number of points
       offset: vertical baseline value
       amplitude: amplitude above (or below with -ve) the offset
    """
    X=[]
    for point_no in range(0, points):
        if point_no<delay or point_no>delay+rise_time+on_time+fall_time:
            X.append(offset)
        elif point_no<delay+rise_time:
            X.append(offset+amplitude*(point_no-delay)/rise_time)
        elif point_no<delay+rise_time+on_time:
            X.append(offset+amplitude)
        else: X.append(offset+amplitude-amplitude*(point_no-delay-rise_time-on_time)/fall_time)
    return X

### Run Settings

In [5]:
#Settings
plot_=True

##rise, on, fall times, offset and amplitude common settings for both channels
rise_time=40
on_time=10
fall_time=400
offset=0
amplitude=-0.01

delayX =500 #delay in number of points for Y channel
delayY =550 #delay in number of points for Y channel

## Noise Settings
noise_=True
noise_offset, noise_rms = 0, 0.001
fc = 0.8

# name of csv file
folder_prefix = "../generated_files/"
no_files=2
points=2000
fs = 20e9 #sampling rate

### File Generation Loop
.......Add noise option

In [7]:
timestamps = [n*(1/fs) for n in range(0,points)]
#common timestamps for all files and both channels
for file_no in range(0, no_files):
    if(file_no%100==0): print('fileno: ', file_no)

    Y=square(points, delayX, rise_time, on_time, fall_time, offset, amplitude)
    X=square(points, delayY, rise_time, on_time, fall_time, offset, amplitude)

    if(noise_):
        whitenoise=random.normal(noise_offset, noise_rms, points)
        b, a = signal.butter(10, fc, 'low')
        lowpassnoise = signal.filtfilt(b, a, whitenoise)
        noisyX=lowpassnoise+X


        whitenoise=random.normal(noise_offset, noise_rms, points)
        b, a = signal.butter(10, fc, 'low')
        lowpassnoise = signal.filtfilt(b, a, whitenoise)
        noisyY=lowpassnoise+Y
    
    if(plot_==True):
        plt.figure(0)
        plt.plot(timestamps, Y)
        plt.plot(timestamps, X)
        plt.title('Signal without Noise')

        plt.figure(file_no+1)
        plt.plot(timestamps, noisyX)
        plt.plot(timestamps, noisyY)
        plt.title('Signal with Noise'+str(file_no+1))
            
    filename=folder_prefix+'genX'+str(file_no).zfill(5)+'.csv'
    # writing to csv file 
    with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        for _ in range(5): csvwriter.writerow(['a', 'a', 'a', 'a', 'a', 'a']) 

        # writing the data rows 
        csvwriter.writerows([[timestamps[n], noisyX[n]] for n in range(points)])
        
    filename=folder_prefix+'genY'+str(file_no).zfill(5)+'.csv'
    # writing to csv file 
    with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        for _ in range(5): csvwriter.writerow(['a', 'a', 'a', 'a', 'a', 'a']) 

        # writing the data rows 
        csvwriter.writerows([[timestamps[n], noisyY[n]] for n in range(points)])

fileno:  0
fileno:  100
fileno:  200
fileno:  300
fileno:  400
fileno:  500
fileno:  600
fileno:  700
fileno:  800
fileno:  900


In [ ]:


for file_no in range(no_files):
    
    
    filename=folder_prefix+str(file_no).zfill(5)+'.csv'
    # writing to csv file 
    with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        for _ in range(5): csvwriter.writerow(['a', 'a', 'a', 'a', 'a', 'a']) 

        # writing the data rows 
        csvwriter.writerows([[timestamps[n], Y[n]] for n in range(points)])

In [ ]:
import csv 
# name of csv file 
folder_prefix = "../generated_files/genX"
no_files=1000

for file_no in range(no_files):
    
    
    filename=folder_prefix+str(file_no).zfill(5)+'.csv'
    # writing to csv file 
    with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        for _ in range(5): csvwriter.writerow(['a', 'a', 'a', 'a', 'a', 'a']) 

        # writing the data rows 
        csvwriter.writerows([[timestamps[n], X[n]] for n in range(points)])

### Checking the Spectrum of Final Signals

In [ ]:
### Checking the spectrum of the final signals

fs=40e9
f, Pxx_den = signal.welch(noisyX, fs, nperseg=1024)

plt.semilogy(f, Pxx_den)

#plt.ylim([0.5e-3, 1])

plt.xlabel('frequency [Hz]')

plt.ylabel('PSD [V**2/Hz]')

plt.show()

fs=40e9
f, Pxx_den = signal.welch(noisyY, fs, nperseg=1024)

plt.semilogy(f, Pxx_den)

#plt.ylim([0.5e-3, 1])

plt.xlabel('frequency [Hz]')

plt.ylabel('PSD [V**2/Hz]')

plt.show()